1.- Importing libraries


In [7]:
import pandas as pd
import numpy as np
import os

2.- Importing Data 

In [17]:
# Folder path

path = r'C:\Users\cmfag\OneDrive\Desktop\Data Analyst Specialist\Achievement 4\4.3_orders_products\02 Data\Prepared Data'

# Importing dataset

df_prods= pd.read_csv(os.path.join(path, 'products.csv'), index_col = False)
df_ords= pd.read_csv(os.path.join(path, 'orders_wrangled.csv'), index_col = False)

3.- Data Consistency Checks

In [20]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


order_id, user_id, and order_number: no discrepances found.


orders_day_of_week: The range for orders_day_of_week is 0 to 6, which aligns with days of the week if indexed from 0 (Sunday) to 6 (Saturday). No further investigation is needed for this field.

order_hour_of_day: The range for order_hour_of_day is from 0 to 23, which is expected for hours in a day. However, the distribution (with a mean around 13 and quartiles from 10 to 16) could indicate peak shopping hours in the afternoon, which aligns with typical customer behavior. No apparent issues, but further exploration of ordering patterns by time of day might be interesting for analysis.

days_since_prior_order: The count for days_since_prior_order is lower than for other columns (3,214,874 versus 3,421,083), indicating that some values are missing. This could be expected for customers placing their first order, as there would be no prior order to calculate a difference.

3.- Check for mixed-type data in df_ords dataframe

In [61]:
df_test = pd.DataFrame() # #Creating a test dataframe
df_test['mix'] = ['a','b',1, True] #Creating a mixed type column

df_test.head() #verifying the result

,mix
0,a
1,b
2,1
3,True


In [73]:
# checking mixed-type data 

for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)
else :
    print ("No mixed data")

mix
No mixed data


C:\Users\cmfag\AppData\Local\Temp\ipykernel_37144\3004091577.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)


4.- Checking for missing values in df_ords dataframe.

In [81]:
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

The results show missing values only in the days_since_prior_order column, with a total of 206,209 missing values. All other columns have zero missing values.
The missing values in days_since_prior_order likely occur because this column represents the number of days since a customer’s previous order. For users placing their first order, there is no prior order to calculate this interval, which would explain why these values are missing. In these cases, it's logical to expect a missing entry, as there isn’t a prior order to compare against.

Proposed Next Steps

Step 1.- Checking the frequency of missing values is the same as the frequency of order_number=1.

Step 2.- Create a new column that flags new orders (True or False).



Adressing the Missing Values

In [108]:
#Step1. 1.1 Create a subset containing only missing values

df_nan = df_ords[df_ords['days_since_prior_order'].isnull() == True]
df_nan

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,0,2539329,1,prior,1,2,8,NaN
11,11,2168274,2,prior,1,2,11,NaN
26,26,1374495,3,prior,1,1,14,NaN
39,39,3343014,4,prior,1,6,11,NaN
45,45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...,...
3420930,3420930,969311,206205,prior,1,4,12,NaN
3420934,3420934,3189322,206206,prior,1,3,18,NaN
3421002,3421002,2166133,206207,prior,1,6,19,NaN
3421019,3421019,2227043,206208,prior,1,1,15,NaN


In [102]:
#Step1. 1.1 result: amount of order_number=1 in df_nan are: 206209

In [104]:
#Step 1. 1.2 Check the frequency of order_number=1 in df_ords

df_ords['order_number'].value_counts()

order_number
1      206209
2      206209
3      206209
4      206209
5      182223
        ...  
96       1592
97       1525
98       1471
99       1421
100      1374
Name: count, Length: 100, dtype: int64

In [106]:
# missing values 206,209 is the same amount as of order_number=1.

In [114]:
#  determine if the order in question is a first order or not
def is_first_order(row):
    if row['order_number'] == 1:
        return True
    else:
        return False

In [117]:
# Apply the is_first_order function to every row of the df_ords dataframe
df_ords.apply(lambda row: is_first_order(row), axis=1)

0           True
1          False
2          False
3          False
4          False
           ...  
3421078    False
3421079    False
3421080    False
3421081    False
3421082    False
Length: 3421083, dtype: bool

In [126]:
#Step 2.- Create a new column that flags new orders (True or False)


df_ords['first_order']=df_ords.apply(lambda row: is_first_order(row), axis=1)
df_ords.head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order
0,0,2539329,1,prior,1,2,8,NaN,True
1,1,2398795,1,prior,2,3,7,15.0,False
2,2,473747,1,prior,3,3,12,21.0,False
3,3,2254736,1,prior,4,4,7,29.0,False
4,4,431534,1,prior,5,4,15,28.0,False


Now is possible to understand if the values are o not missing values.

7.- Checking for duplicate values in your df_ords data

In [134]:
df_ords_dups = df_ords[df_ords.duplicated()]

df_ords_dups

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order


no duplicated values were found. 

Exporting Data

In [140]:
df_ords.to_csv(os.path.join(path,'orders_checked.csv'))